## Perform functional connectivity analysis on trial data, eyes open, and eyes closed data for all viable CLBP subjects
#### Input: *stc.mat
####  Output: con, visualization

In [1]:
import os 
from pathlib import Path
import sys
sys.path.append("/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/")
from eeg_toolkit import utils, preprocess
from eeg_toolkit.functional_connectivity import *

SyntaxError: 'continue' not properly in loop (functional_connectivity.py, line 98)

In [2]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

data_dir = Path("../../../Data")
processed_data_path = data_dir / "Processed_Data"
EC_resting_data_path = data_dir / "STC_EC_resting"
EO_resting_data_path = data_dir / "STC_EO_resting"
zscored_epochs_data_path = data_dir / "STC_zscored_Epochs" / time_win_path

EC_resting_save_path = data_dir / "FC_EC_resting"
EO_resting_save_path = data_dir / "FC_EO_resting"
zscored_epochs_save_path = data_dir / "FC_zscored_Epochs" / time_win_path

save_paths = [EC_resting_save_path, EO_resting_save_path, zscored_epochs_save_path]

[os.makedirs(save_paths) for path in save_paths if not os.path.exists(path)]

[-2.5,0.0,2.5]


[]

In [3]:
# viable subjects
sub_ids = utils.import_subs(data_dir, "sub_ids.txt")
sub_ids = sub_ids[7:]  # select only 64ch subs and turn into set
# print(sub_ids)

EXCLUDE_SUBS = [
    # Chronic Pain Subjects
    '018', '024', '030', '031', '033', '034', '037', 
    '038', '040', '045', '049', '052', '054', '057', '058',
    # Control Subjects
    'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C12', 'C13',
    'C14', 'C16', 'C18', 'C19', 'C20', 'C21', 'C24', 'C26'
    ]

SKIP_NANS = ['018','021','C1.','C2.','C3.']
# TODO: these are just for the grant figures
sub_ids = [sub for sub in sub_ids if sub not in EXCLUDE_SUBS]
sub_ids = [sub for sub in sub_ids if sub not in SKIP_NANS]
print(sub_ids)

['020', '022', '023', '027', '029', '032', '035', '036', '039', '041', '042', '043', '044', '046', '048', '050', '051', '053', '055', '056', 'C10', 'C11', 'C15', 'C17', 'C22', 'C25', 'C27']


In [4]:
sub_ids_CP = [el for el in sub_ids if el.startswith('0')]
sub_ids_HC = [el for el in sub_ids if el.startswith('C')]

print(f"Chronics: {len(sub_ids_CP)}")
print(f"Controls: {len(sub_ids_HC)}")
print(f"Total: {len(sub_ids)}")

Chronics: 20
Controls: 7
Total: 27


In [5]:
# Get stc only from selected labels
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

Freq_Bands = {
    # 'delta': [0, 4],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta": [13.0, 30.0],
    "low-gamma": [30.0, 58.5],
    "notch": [58.5, 61.5],
    "high-gamma": [61.5, 100.0],
}

sfreq = 400  # Hz
tmin, tmax = 0.0, 1.0  # exclude the baseline period for connectivity estimation

con_methods = [
    "wpli",
    "dpli",
    "plv",
    # "",
]

In [ ]:
con_all_epochs_CP = []
con_all_eo_CP = []
con_all_ec_CP = []

for sub_id in sub_ids_CP:
    # First load in STC data and separate epochs by stimulus
    
    # Epochs
    sub_data_path_zepo = os.path.join(zscored_epochs_data_path, sub_id)
    stc_array = load_stc_epochs(zscored_epochs_data_path)
    hand_All_Stim_epochs, back_All_Stim_epochs, pain_ratings, stim_labels = separate_epochs_by_stim(sub_id,
                                                                                                    processed_data_path, 
                                                                                                    stc_array,
                                                                                                    )


    
    for method in con_methods:   
        for band in Freq_Bands:    
            con_epochs = compute_connectivity_epochs(
                sub_data_path_zepo, roi_names, method, Freq_Bands, sfreq
            )
    
    # Resting state
    # eyes closed
    condition = "EO"  # replace with the actual condition
    sub_data_path_EO = f"{EO_resting_data_path}/{time_win_path}/{sub_id}/"
    con_eo = compute_connectivity_resting_state(sub_data_path_EO, roi_names, con_methods, freq_bounds, sfreq, condition)

    # eyes open
    condition = "EC"  # replace with the actual condition
    sub_data_path_EC = f"{data_path_EC}/{time_win_path}/{sub_id}/"
    con_ec = compute_connectivity_resting_state(sub_data_path_EC, roi_names, con_methods, freq_bounds, sfreq, condition)

    con_all_epochs.append(con_epochs)
    con_all_eo.append(con_eo)
    con_all_ec.append(con_ec)

OSError: SourceEstimate File(s) not found for: '../../../Data/STC_Z_scored_Epochs/018.stc'

In [ ]:
# Plot averaged data
plot_connectivity_circle(con_epochs_array, con_methods, Freq_Bands, roi_names)
plot_global_connectivity(epochs, tmin, tmax, n_connections, con_epochs, Freq_Bands)
plot_connectivity(con_epochs, t_con_max, roi_names, con_methods)
plot_con_matrix(con_data, n_con_methods, connectivity_methods, roi_names, foi)
